In [2]:
import time
import numpy as np
from math import pi
from scipy import optimize
import matplotlib.pyplot as plt

N, N_k= 20, 10 #Select the number of points given to calculated the approximated integral in each dimension
               #(N_k refers to number of Matsubara frequencies)

def integral_1r(m): #Tadpole integral
    
    def g_1(k,w):
        return 1/(2*pi)**2*1/(w**2+4*np.sin(k/2)**2+m) 
    
    v1=np.repeat(np.linspace(-pi,pi,N_k),2*N)
    v2=np.tile(np.linspace(-2*pi*T*N,2*pi*T*N,2*N),N_k)
    
    return sum(g_1(v1,v2))*(2*pi*T)*(2*pi/N_k)

def integral_2r(m): #Sunrise integral
    
    def g_2(k1,k2,w1,w2):
        return 1/(2*pi)**4*1/((w1**2+4*np.sin(k1/2)**2+m)*(w2**2+4*np.sin(k2/2)**2+m)*((w1+w2)**2+(2*np.sin(k1/2)+2*np.sin(k2/2))**2+m))
    
    b1=np.repeat(np.linspace(-pi,pi,N_k),N_k*(2*N)*(2*N))
    b2=np.tile(np.repeat(np.linspace(-pi,pi,N_k),(2*N)*(2*N)),N_k)
    b3=np.tile(np.repeat(np.linspace(-2*pi*T*N,2*pi*T*N,2*N),(2*N)),N_k*N_k) 
    b4=np.tile(np.linspace(-2*pi*T*N,2*pi*T*N,2*N),(2*N)*N_k*N_k) 
    
    return sum(g_2(b1,b2,b3,b4))*(2*pi*T)*(2*pi*T)*(2*pi/N_k)*(2*pi/N_k)
    
def integral_3r(m): #Coupling costant renormalisation
    def g_3(k,w):
        return 1/(2*pi)**2*1/(w**2+4*np.sin(k/2)**2+m)**2
    v1=np.repeat(np.linspace(-pi,pi,N_k),2*N)
    v2=np.tile(np.linspace(-2*pi*T*N,2*pi*T*N,2*N),N_k)
    
    return sum(g_3(v1,v2))*(2*pi*T)*(2*pi/N_k)

#Self-consistenet equations to second order in loop expansion

def ecuacion_1r(m):
    return m-(lambd/2)*integral_1r(m)

def ecuacion_2r(m):
    return -mr2+m-(lambd**2/6)*integral_2r(m)

def lambda_bare(m):
    return lambd+(3*lambd**2/2)*integral_3r(m)
    
#m_r1=optimize.newton(ecuacion_2r,m0,tol=1e-2,full_output=True)

Critical line plots

In [31]:
index_lambdas=20
index_mr2=10

T=2 #select the temperature
lambdas=np.linspace(1e-10,15,index_lambdas)
m0=0.1 #educated guess
MR2=[0] #MR2=linspace(0.0001,2,index_mr2) for values of mr2 which are not in the critical region
m_0C,lambda_0 =np.empty([index_lambdas,index_mr2]), np.empty([index_lambdas,index_mr2])

k,l=0,0
for mr2 in MR2:
    print(k)
    for lambd in lambdas:
        m_r1=optimize.newton(ecuacion_2r,m0,tol=1e-2)
        print(m_r1)
        m_0C[l,k]=ecuacion_1r(m_r1)
        lambda_0[l,k]=lambda_bare(m_r1)
        l+=1
    k+=1
    l=0
    
plt.plot(m_0C[:,0], lambda_0[:,0]) #Change the array index if more than one mr2 values are given
plt.xlabel('$m_0^{2}$')
plt.ylabel('$\lambda_0$')
plt.show()

0
0.005971421552537079
0.005976039577168389
0.005973984231339978
0.005978846525879811
0.005976760469598654
0.005980441024241337
0.005978539661759274
0.005979043174228268
0.005979736639967703
0.005978890873565469
0.0059792370454205546
0.005979703139545673
0.0059797314188212046
0.00597904338556572
0.005979043176629228
0.005979172002523335
0.005979085962175284
0.005979391706809233
0.0059792409123876605
0.0059794237410223


Contour plots

In [76]:
index_lambdas=5 #select the number of points accoirding to the dessired precission
index_mr2=5
index_T=20

lambdas=np.linspace(1e-4,1e-3,index_lambdas)
MR2=np.linspace(1e-10,0.001,index_mr2) #MR2=[0]
t=np.linspace(0,2*1e-1,index_T)

m_0C,lambda_0 = np.empty([index_lambdas,index_mr2,index_T]), np.empty([index_lambdas,index_mr2,index_T])

k, l, j= 0, 0, 0
iteraciones=0

startTime = time.time()
for T in t:
    for mr2 in MR2:
        m0=mr2
        for lambd in lambdas:
            print('Completed (%):',100*iteraciones/(index_lambdas*index_mr2*index_T),end='\r')
            m_r1=optimize.newton(ecuacion_2r,m0,tol=1e-2)
            m0=m_r1
            m_0C[l,k,j]=ecuacion_1r(m_r1)
            lambda_0[l,k,j]=lambda_bare(m_r1)
            iteraciones+=1
            l+=1
        k+=1
        l=0
    j+=1
    k=0

executionTime=(time.time()-startTime)
print('Execution time:' ,executionTime)

Execution time: 25.106580018997192


In [77]:
lambda_fijo=10e-4 #select the dessired lambda value to generate a contourplot from the previous grid
m_0C_new, MR2_new, t_new= np.empty([index_mr2,index_T]), np.empty([index_mr2,index_T]), np.empty([index_mr2,index_T])

for i in range(index_mr2):
    for j in range(index_T):
        diferencias=abs(lambda_0[:,i,j]-lambda_fijo)
        index_lambda_fijo=np.where(diferencias==min(diferencias))
        m_0C_new[i,j]=m_0C[index_lambda_fijo,i,j]
    MR2_new[i,:]= MR2[i]#1/MR2[i]

for i in range(index_T):
    t_new[:,i]=t[i]
    
%matplotlib qt
plt.contourf(m_0C_new,t_new,MR2_new,100,cmap="plasma")
plt.xlabel('$m_0^{2}$')
plt.ylabel('$T$')
#plt.xlim()
#plt.ylim()
plt.grid()
cbar=plt.colorbar()
cbar.set_label('${m_{R2}}^2$')